# KMeans

In [ ]:
import numpy as np
import pandas as pd
import time

### Create synthetic dataset

In [ ]:
n_rows = 1000000
n_cols = 500
n_clusters_data = 200
cluster_std = 1.0
# our transform code doesn't yet work with np.float32
#dtype='float32'
dtype='float64'
from cuml.datasets import make_blobs
data, _ = make_blobs(
        n_rows, n_cols, n_clusters_data, cluster_std=cluster_std, random_state=0, dtype=dtype
    )  # make_blobs creates a random dataset of isotropic gaussian blobs.

data = data.get()

### Convert dataset to Spark DataFrame

In [ ]:
# pd_data = pd.DataFrame({"features": list(data)})
# df = spark.createDataFrame(pd_data)
df = sc.parallelize(list(data)).map(lambda x: [x.tolist()]).toDF(["features"])

## Spark-RAPIDS ML

In [ ]:
from spark_rapids_ml.clustering import KMeans

In [ ]:
num_workers = 2
n_clusters = 200
max_iter = 30

In [ ]:
gpu_kmeans = KMeans(num_workers=num_workers,tol=0.0)\
            .setK(n_clusters)\
            .setFeaturesCol("features")\
            .setMaxIter(max_iter)

Estimator can be persisted and reloaded.

In [ ]:
estimator_path = "dbfs:/tmp/sparkcuml-kmeans-estimator"

In [ ]:
gpu_kmeans.write().overwrite().save(estimator_path)
gpu_kmeans_loaded = KMeans.load(estimator_path)

### Fit

In [ ]:
start_time = time.time()
gpu_model = gpu_kmeans_loaded.fit(df)
print(f"Fit took: {time.time() - start_time} sec")

In [ ]:
gpu_kmeans_loaded.getOrDefault("n_clusters")

In [ ]:
sorted_clusters = sorted(gpu_model.cluster_centers_)

In [ ]:
[vec[0:10] for vec in sorted_clusters[0:2]]

### Transform

In [ ]:
model_path = "dbfs:/tmp/sparkcuml-kmeans-model"

In [ ]:
gpu_model.write().overwrite().save(model_path)

In [ ]:
gpu_model_loaded = gpu_model.read().load(model_path)

In [ ]:
[vec[0:10] for vec in sorted(gpu_model_loaded.cluster_centers_)[0:2]]

In [ ]:
transformed_df = gpu_model_loaded.setPredictionCol("transformed").transform(df)

In [ ]:
transformed_df.printSchema()

In [ ]:
transformed_df.count()

In [ ]:
transformed_df.show(10)

## PySpark ML

In [ ]:
from pyspark.ml.clustering import KMeans

In [ ]:
spark_ml_kmeans = KMeans()\
    .setFeaturesCol("features")\
    .setK(n_clusters)\
    .setTol(0.0)\
    .setMaxIter(max_iter)

Convert array sql type to VectorUDT expected by Spark ML algos

In [ ]:
from pyspark.ml.functions import array_to_vector

In [ ]:
vector_df = df.select(array_to_vector(df.features).alias("features"))

### Fit

In [ ]:
start_time = time.time()
spark_ml_kmeans_model = spark_ml_kmeans.fit(vector_df)
print(f"Fit took: {time.time() - start_time} sec")

In [ ]:
type(spark_ml_kmeans_model.clusterCenters()[0])

In [ ]:
sorted_spark_ml_cluster_centers = sorted([vec.tolist() for vec in spark_ml_kmeans_model.clusterCenters()])
[vec[0:10] for vec in sorted_spark_ml_cluster_centers]

### Transform

In [ ]:
spark_transformed = spark_ml_kmeans_model.setPredictionCol("transformed").transform(vector_df)

In [ ]:
spark_transformed.filter(spark_transformed.transformed >= 0).count()

In [ ]:
spark_transformed.show(10)

## cuML (single-node on driver)

In [ ]:
from cuml import KMeans

In [ ]:
cuml_kmeans = KMeans(n_clusters=200, max_iter=max_iter, tol=0.0, verbose=6)

### Fit

In [ ]:
start_time = time.time()
cuml_model = cuml_kmeans.fit(data)
print(f"Fit took: {time.time() - start_time} sec")

In [ ]:
cuml_model.n_iter_

In [ ]:
cuml_centers_sorted = sorted([vec.tolist() for vec in cuml_model.cluster_centers_])
[vec[0:10] for vec in cuml_centers_sorted[0:2]]